## Project 3 - **Space X Falcon 9 first stage landing prediction**

### Building an Interactive Dashboard with Ploty Dash

In this lab, we will be building a Ploty Dash application for users to perform interactive visual analytics on SpaceX launch data in real time.

This dashboard application contains input componenets such as dropdown list and a range slider to interact with a pie chart and a scatter point chart. You will be guided to build this dashboard application via the following tasks:

- Task 1: Add a lauch site Drop-down Input Componenet
- Task 2: Add a call back function to render `success-pie-chart`, based on selected site dropdown
- Task 3: Add a range slider to Select Payload
- Task 4: Add a callback function to render the `success-payload-scatter-chart` scatter plot.

After visual analysis using the dashboard, we should be able to obtain some insights to answer the following questions.

1. Which site has the largest successful launches?
2. Which site has the highest launch success rate?
3. Which payload ranges has the highest lauch success rate?
4. Which payload range has the lowest launch success rate?
5. Which F9 Booster version (`v1.0`, `v1.1`, `FT`, `B4`, `B5`, etc.,)has the highest launch success rate?

Note - I'm performing this project on the jupyter notebook, alternatively we can perform on diferrent software like clou IDE, sypder python terminal and then use the anaconda navigator to set the link for webpage of our dashboard using the downloaded python script file from sypder.

In [2]:
!pip install pandas numpy plotly dash jupyter-dash

#importing all required libraries 
#Data handling
import pandas as pd
import numpy as np

#visualization
import plotly.express as px
import plotly.graph_objects as go

#dash for interactive dashboard (jupyter version)
from jupyter_dash import JupyterDash
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output

Defaulting to user installation because normal site-packages is not writeable


### Dowmload a skeleton dashboard application and dataset

First, let's get the SpaceX Launch dataset for this project:

In [3]:
import urllib.request

# URLs for dataset and skeleton script
dataset_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
script_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/t4-Vy4iOU19i8y6E3Px_ww/spacex-dash-app.py"

# Download dataset
urllib.request.urlretrieve(dataset_url, "spacex_launch_dash.csv")

# Download skeleton Dash app
urllib.request.urlretrieve(script_url, "spacex-dash-app.py")

print("Files downloaded successfully!")


Files downloaded successfully!


In [4]:
ls

 Volume in drive C is Windows
 Volume Serial Number is BCF3-9EC6

 Directory of C:\Users\ABID\Applied Datascience Capstone Project

01/04/2025  08:35    <DIR>          .
29/03/2025  21:11    <DIR>          ..
31/03/2025  22:24    <DIR>          .ipynb_checkpoints
30/03/2025  18:44            93,104 1. Space X Project - Data Collection.ipynb
30/03/2025  19:43            50,556 2. SpaceX Project - Data Collection Part 2.ipynb
30/03/2025  22:26            39,578 3. SpaceX Project - Data Wrangling.ipynb
30/03/2025  23:39            32,328 4. SpaceX Project - EDA with SQL.ipynb
31/03/2025  00:19           426,810 5. SpaceX Project - with EDA and Matplotlib.ipynb
31/03/2025  20:58           333,889 6. SpaceX Project - Visual Analytics with Folium.ipynb
01/04/2025  08:35            19,915 7. SpaceX Project - Visual Analytics with Ploty Dash.ipynb
30/03/2025  15:44            11,916 dataset_spacex_part_1.csv
30/03/2025  20:21            12,102 dataset_spacex_part_2.csv
31/03/2025  00:19       

In [5]:
import pandas as pd
df = pd.read_csv('spacex_launch_dash.csv')
df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


## Add a Launch site Drop-down Input Componenet

We have four different launch sites and we would like to first see which one has the largest success count. Then, we would like to select one specific site and check its detailed success rate (class = 0 vs class = 1).

As such, we will need a dropdown menu to let us select different lauch sites.

Input attributes:

- `id` attribute with value `site-dropdown`
- `option` attribute is a list of dict-like option objects (with `label` and `value` attributes). We can set the `lable` and `value` all to be the launch site names in the `space_df` and we need to include the default `All` option.
- value` attribute with default dropdown value to be `All` meaning all sites are selected.
- `placeholder` attribute to show text sescription about this input area, such as `Select a Launch Site here`.
- `searchable` attribute to be True so we can enter keywords to search launch sites.

In [6]:
#define launch site options
launch_sites = [
    {'label': 'All', 'value': 'All'},
    {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
    {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
    {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
    {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'}
]

#create dropdown component
launch_site_dropdown = dcc.Dropdown(
    id = 'site-dropdown',
    options = launch_sites,
    value = 'ALL',   #Default selection
    placeholder = 'Select a launch site here',
    searchable = True #enable search functionality
)

## Add a callback function to render `success-pie-chart` based on selected site dropdown

The general idea of this call back function is to get the selected launch site from `site-dropdown` and render a pie chart visualizing launch success counts.

Dash callback function is a type of Python function which will be automatically called by Dash whenever receiving an input componenet updates, such as a click or dropdown selecting event.

let's add a callback function in `spacex_dash_app.py` (python script) including the following aplication logic:
- Input is set to be the `site-dropdown` dropdown, i.e., `Input(componenet_id = 'site-dropdown', component_property = 'value')
- Output to be the graph with ID `success-pie-chart`, i.e., `Output(component_id = 'success-pie-chart', component_property = 'figure')`
- A `If-else` statement to check if All sites were slected or just a specific launch site was selected.
  - If All sites are selected, we will use all rows in the dataframe `space_df` to render and return a pie chart graph to show the total success launches (i.e., the total count of `class` column)
  - If a specific launch site is selected, we need to filter the dataframe `spacex_df` first in order to include the only data for the selected site. Then, render and return a pie chart graph to show the success (`class=1`) count and failed (`class=0`) count for the selected site.

In [12]:
from dash.dependencies import Input, Output

#load the dataset
spacex_df = pd.read_csv('spacex_launch_dash.csv')

#list of unique launch sites
launch_sites = spacex_df['Launch Site'].unique()
options = [{'label': site, 'value': site} for site in launch_sites]
options.insert(0, {'label': 'All sites', 'value': 'ALL'})

#Initialize the dash app
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1('SpaceX Launch Success Dashboard', style = {'textAlign': 'centre'}),

    # Dropdown menu
    dcc.Dropdown(
        id = 'site-dropdown',
        options = options,
        value = 'All',
        placeholder = 'Select a Launch Site here',
        searchable = True
    ),

    html.Br(),

    #pie chart
    dcc.Graph(id = 'success-pie-chart'),
])

#callback to update pie chart
@app.callback(
    Output(component_id = 'success-pie-chart', component_property = 'figure'),
    Input(component_id = 'site-dropdown', component_property = 'value')
)
def update_pie_chart(selected_site):
    if selected_site == 'All':
        fig = px.pie(spacex_df, value = 'class', names = 'Launch Site', title = 'Total Success Launches for All Sites')

    else: 
        filtered_df = spacex_df[spacex_df['Launch Site'] == selected_site]
        fig = px.pie(filtered_df, names = 'class', title = f'success vs. Failure for {selected_site}')
    return fig

## 3. Add a Range Slider to Select Payload

Next, we want to find if variable payload is correlated to mission outcome. From a dashboard point of view, we want to able to easily select different payload range and see if we can identify some visual patterns.

Find and complete a commented input with the following attribute:
- `id` to be `payload-slider`
- `min` indicating the slider starting point, we set its value to be 0 (kg)
- `max` indicating the slider ending point to, we set its value to be 10000 (kg)
- `step` indicating the slider interval on the slider, we set its value to be 1000 (kg)
- `value` indicating the current selected range, we could set it to be `min_payload` and `max_payload`.

In [14]:
#define default payload range values
min_payload = 0
max_payload = 10000

#add range slider to the dashboard
dcc.RangeSlider(
    id = 'payload-slider',
    min = 0,
    max = 10000,
    step = 1000,
    marks = {i: str(i) for i in range (0, 11000, 2000)}, #label every 2000kg
    value = [min_payload, max_payload] # default range selection
)

RangeSlider(min=0, max=10000, step=1000, marks={0: '0', 2000: '2000', 4000: '4000', 6000: '6000', 8000: '8000', 10000: '10000'}, value=[0, 10000], id='payload-slider')

## 4. Add a callback function to render the `success-payload-scatter-chart` scatter plot

Next, we want to plot a scatter plot with the x axis to be the payload and the y axis to be the launch outcome (i.e., `class` column).

As such, we can visually observe how payload may be correlated with mission outcomes for selected site(s).

Inaddition, we want to color-label the Booster version on each scatter point so that we may observe mission outcomes with different boosters.

Now, let's add a call function including the following application logic:
- Input to be `[Input(component_id = 'site-dropdown', component_property = 'value'), Input(component_id = 'payload-slider', component-property = 'value')]`, note that we have two input components, one to recive selected launch site and other to recive selected payload range.
- Output to be `Output(component_id = 'success-payload-scatter-chart', component_property = 'figure')`
- A `If-Else` statement to check if All sites were selected or just a specific launch site was selected
  - If All sites are selcted, render a scatter plot to display all values for variable `Payload Mass (kg)` and variable `class`.
  - In addition, the point color need to be set to the booster version i.e., `color = 'Booster Version Category'`
  - If a specific launch site is selected, we need to filter the `spacex_df` first, and render a scatter chart to show values `Payload Mass (kg)` and `class` for the selected site, and color-label the point using `Booster Version Category` likewise.

Final Dash app code 
- Dropdown for selecting launch sites
- Pie chart for success rate visualization
- Range slider for payload selection
- Scatter plot for payload vs launch outcome

In [23]:
#full code for spacex_dash_app.py
#import required libraries
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

#read the spacex dataset
spacex_df = pd.read_csv('spacex_launch_dash.csv')

#etract unique launch sites
launch_sites = spacex_df['Launch Site'].unique()
launch_sites_options = [{'label': site, 'value': site} for site in launch_sites]
launch_sites_options.insert(0, {'label': 'All Sites', 'value': 'ALL'})

#define min and max payload values
min_payload = spacex_df['Payload Mass (kg)'].min()
max_payload = spacex_df['Payload Mass (kg)'].max()

#initialize the dash app
app = dash.Dash(__name__)

#define app layout
app.layout = html.Div(children = [
    html.H1('SpaceX Launch Records Dashboard',
            style = {'textAlign': 'center', 'color': '#503D36', 'font-size': 40}),

    #dropdown for lauch site selection
    html.Div([
        html.Label("Select Launch Site:"),
        dcc.Dropdown(
        id = 'site-dropdown',
        options = launch_sites_options,
        value = 'ALL',
        placeholder = 'Select a Launch Site here',
        searchable = True
    ),
    ], style={'width': '50%', 'margin': 'auto'}),

    html.Br(),

    # Pie chart to show success counts
    html.Div(dcc.Graph(id='success-pie-chart')),

    html.Br(),

    #payload range slider
    html.Div([
        html.Label("Select Payload Range (kg):"),
        dcc.RangeSlider(
        id = 'payload-slider',
        min = 0,
        max = 10000,
        step = 1000, marks = {i: str(i) for i in range (0, 11000, 2000)},
        value = [min_payload, max_payload]
    ),
    ], style={'width': '70%', 'margin': 'auto'}),
    html.Br(),

    #scatter plot for payload vs success
    html.Div(dcc.Graph(id = 'Success-Payload-Scatter-Chart')),
])

#callback function for updating the success pie chart
@app.callback(
    Output('Success-Pie-Chart', 'figure'),
    [Input('site-dropdown', 'value')]
)
def update_pie_chart(selected_site):
    if selected_site == 'ALL':
        #Pie chart showing totla success launches
        success_counts = spacex_df.groupby('Launch Site')['class'].sum().reset_index()
        fig = px.pie(
            success_counts,
            name = 'Launch Site',
            values = 'class',
            title = 'Total Success Launches for All Sites'
        )
    else:
        #pie chart for selected site (success vs Failure)
        site_df = spacex_df[spacex_df['Launch Site'] == selected_site]
        success_counts = site_df['class'].value_counts().reset_index()
        success_counts.columns = ['class', 'count']
        fig = px.pie(
            success_counts,
            names = 'class',
            values = 'count',
            title = f'Success Rate for {selected_site}'
        )
        return fig

#callback function for updating the scatter plot
@app.callback(
    Output('success-payload-scatter-chart', 'figure'),
    [Input('site-dropdown', 'value'),
     Input('payload-slider', 'value')]
)
def update_scatter_chart(selected_site, payload_range):
    #filter data by payload range
    filtered_df = spacex_df[
    (spacex_df['Payload Mass (kg)'] >= payload_range[0]) &
    (spacex_df['Payload Mass (kg)'] <= payload_range[1])
    ]

    if selecte_site == 'ALL':
        fig = px.scatter(
            filtered_df,
            x='Payload Mass (Kg)',
            y = 'class',
            color = 'Booster Version Category',
            title = 'Payload vs Launch Outcome for All Sites',
        )
    else:
        site_df = filtered_df[filtered_df['Launch Site'] == selected_site]
        fig = px.scatter(
            site_df,
            x = 'Payload Mass (kg)',
            y = 'class',
            color = 'Booster Version Category',
            title = f'Payload vs Launch Outcome for {selected_site}',
        )
    return fig

# Run the Dash app
if __name__ == '__main__':
    app.run(debug = True)
           
        
        
        

OSError: Address 'http://127.0.0.1:8050' already in use.
    Try passing a different port to run.

## Observations

### 1. Which site has the largest successful launches?
To find this, we need to check whichlaunch site has the highest count of `class = 1` (success launches)

In [24]:
spacex_df[spacex_df['class'] == 1]['Launch Site'].value_counts()

Launch Site
KSC LC-39A      10
CCAFS LC-40      7
VAFB SLC-4E      4
CCAFS SLC-40     3
Name: count, dtype: int64

### 2. Which site has the highest launch success rate?
Success rate = (successful launches/Total Launches at site)*100

In [25]:
spacex_df.groupby('Launch Site')['class'].mean()*100

Launch Site
CCAFS LC-40     26.923077
CCAFS SLC-40    42.857143
KSC LC-39A      76.923077
VAFB SLC-4E     40.000000
Name: class, dtype: float64

### 3. Which Pyload ranges(s) has the highest launch and lowest success rate?
To determine this, group the dataset by payload ranges and calculate the success rate

In [27]:
spacex_df.groupby(pd.cut(spacex_df['Payload Mass (kg)'], bins = [0, 2000, 4000, 6000, 8000, 10000]))['class'].mean()*100

C:\Users\ABID\AppData\Local\Temp\ipykernel_2440\3424568621.py:1: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



Payload Mass (kg)
(0, 2000]        27.272727
(2000, 4000]     61.904762
(4000, 6000]     38.461538
(6000, 8000]      0.000000
(8000, 10000]    60.000000
Name: class, dtype: float64

### 4. Which F9 Booster version has the highest launch success rate?
Group by Booster Version and calculate the success rate:

In [28]:
spacex_df.groupby('Booster Version')['class'].mean()*100

Booster Version
F9 B4  B1039.2      0.0
F9 B4  B1040.2      0.0
F9 B4  B1041.2      0.0
F9 B4  B1043.2      0.0
F9 B4 B1039.1     100.0
F9 B4 B1040.1     100.0
F9 B4 B1041.1     100.0
F9 B4 B1042.1     100.0
F9 B4 B1043.1     100.0
F9 B4 B1044         0.0
F9 B4 B1045.1     100.0
F9 B5  B1046.1    100.0
F9 FT  B1021.2    100.0
F9 FT  B1029.2    100.0
F9 FT  B1031.2    100.0
F9 FT  B1032.2      0.0
F9 FT  B1035.2    100.0
F9 FT  B1036.2      0.0
F9 FT  B1038.2      0.0
F9 FT B1019       100.0
F9 FT B1020         0.0
F9 FT B1021.1     100.0
F9 FT B1022       100.0
F9 FT B1023.1     100.0
F9 FT B1024         0.0
F9 FT B1025.1     100.0
F9 FT B1026       100.0
F9 FT B1029.1     100.0
F9 FT B1030         0.0
F9 FT B1031.1     100.0
F9 FT B1032.1     100.0
F9 FT B1034         0.0
F9 FT B1035.1     100.0
F9 FT B1036.1     100.0
F9 FT B1037         0.0
F9 FT B1038.1     100.0
F9 v1.0  B0003      0.0
F9 v1.0  B0004      0.0
F9 v1.0  B0005      0.0
F9 v1.0  B0006      0.0
F9 v1.0  B0007      0.0


F9 B5 - 100% success rate - B1046.1 (100% success)